In [1]:
from base_dataset import BaseDataset, MattingTransform

import numpy as np
from PIL import Image
import cv2

/home/ubuntu/miniconda3/envs/hrnet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = '/home/ubuntu/data/workspace/deeplabv3_plus/people_segmentation'
train_transform = MattingTransform()
ds = BaseDataset(root_dir, image_dir="images", mask_dir="masks", transform=train_transform)
# ds = BaseDataset(root_dir)

5678


In [3]:
sample = ds[4]

In [4]:
fn, img, trimap, mask = sample

In [5]:
img.shape

torch.Size([3, 512, 512])

In [6]:
mask.shape

torch.Size([1, 512, 512])

In [7]:
trimap.shape

torch.Size([512, 512, 1])

In [ ]:
fn

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(img, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(mask, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [ ]:
Image.fromarray(np.array(np.squeeze(np.transpose(trimap, (1, 2, 0))).numpy() * 255.0, dtype=np.uint8))

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
from scipy.ndimage import morphology
import numpy as np
from PIL import Image
import cv2

In [16]:
torch_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [9]:
class ImagesDataset(Dataset):
    def __init__(self, root, transform=None, w=1024, h=576):
        self.root = root
        self.transform = transform
        self.tensor = transforms.Compose([transforms.ToTensor()])
        self.w = w
        self.h = h
        self.imgs = sorted([os.path.join(self.root, 'images', img) for img in os.listdir(os.path.join(self.root, 'images'))])
        self.alphas = sorted([os.path.join(self.root, 'masks', aph) for aph in os.listdir(os.path.join(self.root, 'masks'))])
        print(len(self.imgs))
        assert len(self.imgs) == len(self.alphas), 'the number of dataset is different, please check it.'


    def getTrimap(self, alpha):
        fg = np.array(np.equal(alpha, 255).astype(np.float32))
        unknown = np.array(np.not_equal(alpha, 0).astype(np.float32))  # unknown = alpha > 0
        unknown = unknown - fg
        unknown = morphology.distance_transform_edt(unknown == 0) <= np.random.randint(10, 20)
        trimap = fg
        trimap[unknown] = 0.5
        return trimap[:, :, :1]

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = cv2.imread(self.imgs[idx])
        alpha = cv2.imread(self.alphas[idx])
        # h, w, c = img.shape
        # rh = 512
        # rw = int(w / h * 512)
        # rh = rh - rh % 32 #512
        # rw = rw - rw % 32 #896    1024 or 576 %32==0
        img = cv2.resize(img, (self.w, self.h))
        alpha = cv2.resize(alpha, (self.w, self.h))
        trimap = self.getTrimap(alpha)
        if self.transform:
            img = self.transform(img)
        alpha = self.tensor(alpha[:, :, 0])
        return self.imgs[idx], img, trimap, alpha

In [17]:
ds = ImagesDataset("/home/ubuntu/data/workspace/deeplabv3_plus/people_segmentation", torch_transforms)

5678


In [18]:
sample = ds[4]

/tmp/ipykernel_2711243/3324219205.py:18: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  unknown = morphology.distance_transform_edt(unknown == 0) <= np.random.randint(10, 20)


In [19]:
fn, img, trimap, alpha = sample

In [20]:
img.shape

torch.Size([3, 576, 1024])

In [21]:
trimap.shape

(576, 1024, 1)

In [22]:
alpha.shape

torch.Size([1, 576, 1024])